<a href="https://colab.research.google.com/github/zilavalencia/ChungaraVZila-IA-SIS420/blob/main/Laboratorios/Laboratorio02/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LABORATORIO 2 REDES NEURONALES USANDO PYTORCH
Estudiantes:

Portillo Mercado Daniela

Chungara Valencia Zila

Rodas Palacios Max Jherzon

Link dataset: https://www.kaggle.com/datasets/jvageesh11/simpsons-mnist




### DATASET:

Presenta imágenes de 28x28 de personajes de Los Simpson
Formato RGB
Incluye 8000 muestras de entrenamiento y 2000 muestras de prueba  
10 clases de personajes.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os
import random

# Definir transformaciones para las imágenes
# Transformación para prueba - redimensionar a 28x28 y convertir a tensor
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Redimensionar todas las imágenes a 28x28
    transforms.ToTensor(),        # Convertir a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar RGB
])

# Transformación para entrenamiento - incluye aumentación de datos
transform_train = transforms.Compose([
    transforms.Resize((28, 28)),  # Redimensionar todas las imágenes a 28x28
    transforms.RandomHorizontalFlip(p=0.5),  # 50% de probabilidad de volteo
    transforms.ToTensor(),        # Convertir a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar RGB
])

# Cargar los datasets
def load_datasets(data_dir):
    train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train2'), transform=transform_train)
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform)

    # Crear dataloaders
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    return train_loader, test_loader, train_dataset, test_dataset

# Definir el modelo MLP
class MLP(nn.Module):
    def init(self, input_size, hidden_sizes, num_classes):
        super(MLP, self).init()

        # Capa de entrada
        layers = [nn.Flatten()]

        # Agregar capas ocultas
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            prev_size = hidden_size

        # Capa de salida
        layers.append(nn.Linear(prev_size, num_classes))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Función para entrenar el modelo
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs):
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    # Bucle de entrenamiento
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            # Mover datos a GPU si está disponible
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward y optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Estadísticas
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calcular pérdida y precisión de entrenamiento
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)

        # Evaluar en el conjunto de prueba
        test_loss, test_acc = evaluate_model(model, test_loader, criterion)
        test_losses.append(test_loss)
        test_accuracies.append(test_acc)

        print(f'Época {epoch+1}/{num_epochs}, '
              f'Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%, '
              f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
        print(f'Época {epoch+1}/{num_epochs}, Train: {epoch_acc:.2f}% ({correct}/{total}), Test: {test_acc:.2f}%, Train Loss: {epoch_loss:.4f}, Test Loss: {test_loss:.4f}')

        # Verificar si cumplimos con los criterios (precisión > 90% y pérdida < 0.05)
        if test_acc > 90 and test_loss < 0.05:
            print(f'¡Criterios cumplidos en la época {epoch+1}!')
            break

    return train_losses, test_losses, train_accuracies, test_accuracies

# Función para evaluar el modelo
def evaluate_model(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            if torch.cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return running_loss / len(dataloader), 100 * correct / total

# Función para probar el modelo con imágenes aleatorias
def test_random_images(model, test_dataset, num_images=10):
    # Obtener índices aleatorios
    indices = random.sample(range(len(test_dataset)), num_images)

    # Configurar el modelo en modo evaluación
    model.eval()

    # Configurar la figura para mostrar imágenes
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.flatten()

    with torch.no_grad():
        for i, idx in enumerate(indices):
            # Obtener imagen y etiqueta real
            image, true_label = test_dataset[idx]

            # Obtener la predicción
            if torch.cuda.is_available():
                image = image.cuda()

            output = model(image.unsqueeze(0))
            _, predicted = torch.max(output, 1)

            # Preparar imagen para mostrar
            image = image.cpu().numpy().transpose(1, 2, 0)  # Cambiar de [C,H,W] a [H,W,C] para RGB
            image = (image * 0.5 + 0.5)  # Desnormalizar
            image = image.clip(0, 1)  # Asegurar valores en rango [0,1]

            # Mostrar imagen y predicciones
            axes[i].imshow(image)
            axes[i].set_title(f'Pred: {test_dataset.classes[predicted.item()]}\nTrue: {test_dataset.classes[true_label]}')
            axes[i].axis('off')

    plt.tight_layout()
    plt.show()

    return indices

# Función principal
def main(data_dir, hidden_sizes=[64], num_epochs=50):
    # Cargar datos
    train_loader, test_loader, train_dataset, test_dataset = load_datasets(data_dir)

    # Obtener número de clases
    num_classes = len(train_dataset.classes)
    print(f"Clases detectadas: {train_dataset.classes}")

    # Definir el modelo
    input_size = 3 * 28 * 28  # RGB 28x28 (3 canales)
    model = MLP(input_size, hidden_sizes, num_classes)

    # Mover el modelo a GPU si está disponible
    if torch.cuda.is_available():
        model = model.cuda()
        print("Usando GPU para el entrenamiento")
    else:
        print("Usando CPU para el entrenamiento")

    # Definir pérdida y optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    # Entrenar el modelo
    train_losses, test_losses, train_accuracies, test_accuracies = train_model(
        model, train_loader, test_loader, criterion, optimizer, num_epochs
    )

    # Visualizar las pérdidas y precisiones
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.title('Pérdida por Época')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(test_accuracies, label='Test Accuracy')
    plt.title('Precisión por Época')
    plt.xlabel('Época')
    plt.ylabel('Precisión (%)')
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Probar el modelo con imágenes aleatorias
    print("\nProbando el modelo con 10 imágenes aleatorias:")
    indices = test_random_images(model, test_dataset)

    # Guardar el modelo
    torch.save(model.state_dict(), 'mlp_model.pth')
    print("Modelo guardado como 'mlp_model.pth'")

# Ejecutar el programa
if name == "main":
    # Reemplazar con la ruta a tu directorio de datos
    data_dir = "/home/n3st/Documents/Python/fotos"  # Ajusta esta ruta a tu directorio de datos

    # Puedes ajustar la arquitectura de la red neural aquí
    hidden_sizes = [512, 256, 128, 64]  # Cuatro capas ocultas

    main(data_dir, hidden_sizes, num_epochs=100)